# Análise Eleitoral

### 0) Configuração dos bancos a serem utilizados

In [1]:
import pandas as pd
import tse

In [2]:
CANDIDATO = 'EDUARDO PAES'

In [3]:
database_1 = 'databases/rj_2024.sql'
database_2 = 'databases/eleicoes_2024.sql'

In [4]:
analise = tse.TseAnalysis(
    database={
        'path': database_1,
        'alias': 'rj'
    })

In [5]:
analise.attach_database(
    attach= [
        {
            'path': database_2,
            'alias': 'geral'
        }
    ])

Bancos carregados:
0) main
-path: d:\projects\tse-data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
2) rj
-path: d:\projects\tse-data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
3) geral
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [6]:
analise.create_index(
    table='resultados',
    columns=['NM_VOTAVEL', 'NR_VOTAVEL', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO', 'SG_UF']
)
analise.create_index(
    table='eleitorado',
    columns=['NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO', 'SG_UF']
)

'Indices criados com sucesso'

---

### 1) Criação de tabela com perfil de eleitorado por seção

In [7]:
# a função generate_stats_query foi escrita para gerar uma query específica para a tabela rj. eleitorado
# ela analisa os valores distintos de cada coluna argumento e produz uma query que soma a quantidade de aparições para cada caso
q = analise.generate_stats_query(
    alias='rj',
    table='eleitorado',
    cols=[
        'DS_GRAU_ESCOLARIDADE',
        'DS_RACA_COR',
        'DS_FAIXA_ETARIA',
        'DS_GENERO'
    ]
)

In [8]:
eleitorado_secao = pd.read_sql(q, analise.conn)

In [9]:
eleitorado_secao.head(2) # este DataFrame expressa a quantidade de eleitores por seçao divido em categorias

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_85_a_89_anos,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado
0,58009,1015,255,23,61,23,114,37,42,91,...,10,4,0,1,7,0,0,175,211,0
1,58009,1015,255,24,79,29,99,26,48,83,...,2,0,1,0,10,1,0,183,203,0


In [10]:
q_2 = """
    SELECT
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        QT_ELEITOR_SECAO as eleitores_totais
    FROM
        geral.locais
    WHERE
        SG_UF = 'RJ'
"""

In [11]:
eleitores_secao = pd.read_sql(q_2, analise.conn)

In [12]:
eleitores_secao.head(2) # complementa o DataFrame anterior com a quantidade total de eleitores na seção

,municipio,local,zona,secao,eleitores_totais
0,60011,1252,185,136,406
1,60011,1341,14,164,446


In [13]:
eleitorado_secao = pd.merge(
    eleitorado_secao,
    eleitores_secao,
    how='inner',
    on=[
        'municipio',
        'local',
        'zona',
        'secao'
    ]
)

In [14]:
eleitorado_secao.head(2) # resultado obtido

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,58009,1015,255,23,61,23,114,37,42,91,...,4,0,1,7,0,0,175,211,0,386
1,58009,1015,255,24,79,29,99,26,48,83,...,0,1,0,10,1,0,183,203,0,386


---

### 2) Cria tabela com apuração dos resultados por seção de determinado candidato

In [15]:
query = f"""
    SELECT
        NM_VOTAVEL as candidato,
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        QT_VOTOS as votos_recebidos
    FROM
        rj.resultados
    WHERE
        NM_VOTAVEL LIKE '{'%' + CANDIDATO.replace(' ', '%') + '%'}'
"""

In [16]:
resultado_secao = pd.read_sql(query, analise.conn)

In [17]:
resultado_secao.head(2)

,candidato,municipio,local,zona,secao,votos_recebidos
0,EDUARDO DA COSTA PAES,60011,1317,245,254,170
1,EDUARDO DA COSTA PAES,60011,1341,123,262,155


In [18]:
resultado_perfil = pd.merge(
    resultado_secao,
    eleitorado_secao,
    how='inner',
    on=['municipio', 'local', 'zona', 'secao']
)

In [19]:
resultado_perfil.head(2)

,candidato,municipio,local,zona,secao,votos_recebidos,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,EDUARDO DA COSTA PAES,60011,1317,245,254,170,90,38,60,45,...,4,8,10,0,1,0,263,194,2,459
1,EDUARDO DA COSTA PAES,60011,1341,123,262,155,70,18,100,77,...,7,10,11,0,0,0,239,186,0,425


In [20]:
cols_percentuais = set(resultado_perfil.columns.to_list()) - {'candidato', 'municipio', 'local', 'zona', 'secao'}
cols_percentuais = list(cols_percentuais)

In [21]:
resultado_percentual = resultado_perfil.copy()

In [22]:
resultado_percentual[cols_percentuais] = resultado_perfil[cols_percentuais].div(
    resultado_perfil['eleitores_totais'], 
    axis=0)

In [23]:
resultado_percentual.head(2)

,candidato,municipio,local,zona,secao,votos_recebidos,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,EDUARDO DA COSTA PAES,60011,1317,245,254,0.370370,0.196078,0.082789,0.130719,0.098039,...,0.008715,0.017429,0.021786,0.0,0.002179,0.0,0.572985,0.422658,0.004357,1.0
1,EDUARDO DA COSTA PAES,60011,1341,123,262,0.364706,0.164706,0.042353,0.235294,0.181176,...,0.016471,0.023529,0.025882,0.0,0.000000,0.0,0.562353,0.437647,0.000000,1.0


In [24]:
nome = resultado_percentual['candidato'][0]
resultado_percentual.to_csv(f'resultados_secao_percentuais/{nome.replace(' ', '_')}.csv')